In [1]:
import sys
import time

sys.path.append("..")  # make sure we can run this from the repo
from IPython.display import Markdown, clear_output, display
from ipywidgets import widgets

import cohere
CHAT_MODEL = 'converse-xlarge-nightly'

In [2]:
co = cohere.Client()
start_time = time.time()
streaming_gens = co.chat(query="Hey! Don't worry, 🐝 happy~", max_tokens=20, stream=True)
for i, token in enumerate(streaming_gens):
    print(f"[{i}][{time.time()-start_time:.2f}s] {token}")
# the request id is available after the first token has streamed, and response so far is in texts
streaming_gens.ids, streaming_gens.texts

[0][0.53s] StreamingText(id='b65ddd16-d4d6-4812-8d4c-6d3cb7d18d17', session_id='chat-d489c39a-e152-49da-9ddc-9801bd74d823-26d52712-c07c-4a95-b708-c465a86c808d', index=0, text='I')
[1][0.53s] StreamingText(id=None, session_id=None, index=0, text="'m")
[2][0.54s] StreamingText(id=None, session_id=None, index=0, text=' doing')
[3][0.57s] StreamingText(id=None, session_id=None, index=0, text=' well')
[4][0.60s] StreamingText(id=None, session_id=None, index=0, text=',')
[5][0.63s] StreamingText(id=None, session_id=None, index=0, text=' thank')
[6][0.67s] StreamingText(id=None, session_id=None, index=0, text=' you')
[7][0.68s] StreamingText(id=None, session_id=None, index=0, text='!')
[8][0.71s] StreamingText(id=None, session_id=None, index=0, text=' How')
[9][0.74s] StreamingText(id=None, session_id=None, index=0, text=' about')
[10][0.77s] StreamingText(id=None, session_id=None, index=0, text=' you')
[11][0.80s] StreamingText(id=None, session_id=None, index=0, text='?')


(['b65ddd16-d4d6-4812-8d4c-6d3cb7d18d17'],
 ["I'm doing well, thank you! How about you?"])

## Client example

### Simple Generate Example

In [ ]:
co = cohere.Client()
start_time = time.time()
streaming_gens = co.generate(prompt="Hey! Don't worry, 🐝 happy~", max_tokens=20, stream=True)
for i, token in enumerate(streaming_gens):
    print(f"[{i}][{time.time()-start_time:.2f}s] {token}")
# the request id is available after the first token has streamed, and response so far is in texts
streaming_gens.ids, streaming_gens.texts

[0][0.64s] StreamingText(id='81ab6932-87ee-40dc-8e02-de9a2738c811', index=0, text=' 🐝')
[1][0.67s] StreamingText(id=None, index=0, text='\n')
[2][0.70s] StreamingText(id=None, index=0, text='\n')
[3][0.73s] StreamingText(id=None, index=0, text='1')
[4][0.76s] StreamingText(id=None, index=0, text=')')
[5][0.79s] StreamingText(id=None, index=0, text=' I')
[6][0.83s] StreamingText(id=None, index=0, text=' think')
[7][0.84s] StreamingText(id=None, index=0, text=' this')
[8][0.87s] StreamingText(id=None, index=0, text=' one')
[9][0.90s] StreamingText(id=None, index=0, text=' is')
[10][0.93s] StreamingText(id=None, index=0, text=' the')
[11][0.95s] StreamingText(id=None, index=0, text=' main')
[12][0.98s] StreamingText(id=None, index=0, text=' reason')
[13][1.01s] StreamingText(id=None, index=0, text='.')
[14][1.04s] StreamingText(id=None, index=0, text=' I')
[15][1.06s] StreamingText(id=None, index=0, text="'m")
[16][1.09s] StreamingText(id=None, index=0, text=' afraid')
[17][1.12s] Streami

(['81ab6932-87ee-40dc-8e02-de9a2738c811'],
 [" 🐝\n\n1) I think this one is the main reason. I'm afraid that"])

### Widgets Chat Example

In [ ]:
co = cohere.Client()
max_tokens = 250

text = ""
output_area = display("Type your message below and hit enter to chat!", display_id=True)
input_text = widgets.Text(value="Hey!", description="User:", disabled=False)

def append_text(s):
    global text
    text += s
    output_area.update(Markdown(text))

def stream_response(input_text):
    query = input_text.value
    if query.startswith("<generating"):
        return
    input_text.value = "<generating response>"
    append_text(f"\n\nUser: {query}\n\nChatbot: ")
    streaming_chat = co.chat(
        query=query,
        max_tokens=max_tokens,
        model=CHAT_MODEL,
        stream=True,
    )
    for token in streaming_chat:
        append_text(token.text)
    input_text.value = ""

display(input_text)
input_text.on_submit(stream_response)

## AsyncClient example

### Widgets generate example

In [ ]:
prompt = "안녕하"
num_generations = 5
max_tokens = 50
texts = [f"**Generation #{i+1}:** {prompt}" for i in range(num_generations)]
displays = [display(display_id=True) for t in texts]

async with cohere.AsyncClient() as aio_co:
    start_time = time.time()
    aio_streaming_gens = await aio_co.generate(
        prompt=prompt,
        max_tokens=max_tokens,
        model="command-xlarge-nightly",
        num_generations=num_generations,
        stream=True,
    )

    async for token in aio_streaming_gens:
        texts[token.index] += token.text
        displays[token.index].update(Markdown(texts[token.index]))

### Simple Chat Example

In [ ]:
query = "Do You Want to Build a Snowman?"
async with cohere.AsyncClient() as aio_co:
    streaming_chat = await aio_co.chat(
        query=query,
        max_tokens=50,
        model=CHAT_MODEL,
        stream=True,
    )
    async for token in streaming_chat:
        print(token)
    print(streaming_chat.texts)

In [ ]:
co = cohere.Client()
start_time = time.time()
streaming_gens = co.chat(query="Hey! Don't worry, 🐝 happy~", max_tokens=20, stream=True)
for i, token in enumerate(streaming_gens):
    print(f"[{i}][{time.time()-start_time:.2f}s] {token}")
# the request id is available after the first token has streamed, and response so far is in texts
streaming_gens.ids, streaming_gens.texts

TypeError: <lambda>() missing 2 required positional arguments: 'id' and 'session_id'